In [1]:
from __future__ import print_function, division

from keras.datasets import mnist,fashion_mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GaussianNoise
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers import MaxPooling2D, concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras import losses
from keras.utils import to_categorical
import keras.backend as K
import os
import matplotlib.pyplot as plt

import numpy as np

class BIGAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=['binary_crossentropy'],
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # Build the encoder
        self.encoder = self.build_encoder()

        # The part of the bigan that trains the discriminator and encoder
        self.discriminator.trainable = False

        # Generate image from sampled noise
        z = Input(shape=(self.latent_dim, ))
        img_ = self.generator(z)

        # Encode image
        img = Input(shape=self.img_shape)
        z_ = self.encoder(img)

        # Latent -> img is fake, and img -> latent is valid
        fake = self.discriminator([z, img_])
        valid = self.discriminator([z_, img])

        # Set up and compile the combined model
        # Trains generator to fool the discriminator
        self.bigan_generator = Model([z, img], [fake, valid])
        self.bigan_generator.compile(loss=['binary_crossentropy', 'binary_crossentropy'],
            optimizer=optimizer)


    def build_encoder(self):
        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(self.latent_dim))

        model.summary()

        img = Input(shape=self.img_shape)
        z = model(img)

        return Model(img, z)

    def build_generator(self):
        model = Sequential()

        model.add(Dense(512, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        z = Input(shape=(self.latent_dim,))
        gen_img = model(z)

        return Model(z, gen_img)

    def build_discriminator(self):

        z = Input(shape=(self.latent_dim, ))
        img = Input(shape=self.img_shape)
        d_in = concatenate([z, Flatten()(img)])

        model = Dense(1024)(d_in)
        model = LeakyReLU(alpha=0.2)(model)
        model = Dropout(0.5)(model)
        model = Dense(1024)(model)
        model = LeakyReLU(alpha=0.2)(model)
        model = Dropout(0.5)(model)
        model = Dense(1024)(model)
        model = LeakyReLU(alpha=0.2)(model)
        model = Dropout(0.5)(model)
        validity = Dense(1, activation="sigmoid")(model)

        return Model([z, img], validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0

        for epoch in range(epochs):

            for index in range(nb_batches):
                global_step += 1
                image_batch = X_train[index * batch_size:(index + 1) * batch_size]
                z = np.random.normal(size=(batch_size, self.latent_dim))
                imgs_ = self.generator.predict(z)

                # Select a random batch of images and encode
                # idx = np.random.randint(0, X_train.shape[0], batch_size)
                # imgs = X_train[idx]
                z_ = self.encoder.predict(image_batch)

                # Train the discriminator (img -> z is valid, z -> img is fake)
                d_loss_real = self.discriminator.train_on_batch([z_, image_batch], valid)
                d_loss_fake = self.discriminator.train_on_batch([z, imgs_], fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                # Train the generator (z -> img is valid and img -> z is is invalid)
                g_loss = self.bigan_generator.train_on_batch([z, image_batch], [valid, fake])

                # Plot the progress
                print("epoch:%d step:%d [D loss: %f, acc: %.2f%%] [G loss: %f]" % (epoch,global_step, d_loss[0],
                                                                                   100 * d_loss[1], g_loss[0]))

                # If at save interval => save generated image samples
                if global_step % sample_interval == 0:
                    self.sample_interval(epoch,global_step)


    def sample_interval(self, epoch,global_step):
        r, c = 10, 10
        z = np.random.normal(size=(r*c, self.latent_dim))
        gen_imgs = self.generator.predict(z)

        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        if not os.path.isdir('images_bigan'):
            os.mkdir('images_bigan')
        fig.savefig("images_bigan/epoch_%d_step_%d.png" % (epoch,global_step))
        plt.close()
    def sample_interval_mult(self, epoch):
        r, c = 20, 20
        z = np.random.normal(size=(r*c, self.latent_dim))
        gen_imgs = self.generator.predict(z)

        gen_imgs = 0.5 * gen_imgs + 0.5

        fig1, axs1 = plt.subplots(r, c)
        cnt = 0
        for i in range(r//2):
            for j in range(c//2):
                axs1[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs1[i,j].axis('off')
                cnt += 1
        fig2, axs2 = plt.subplots(r, c)
        for i in range(r//2):
            for j in range(c//2):
                axs2[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs2[i,j].axis('off')
                cnt += 1
        if not os.path.isdir('images_bigan'):
            os.mkdir('images_bigan')
        fig1.savefig("images_bigan/%d_1.png" % epoch)
        fig2.savefig("images_bigan/%d_2.png" % epoch)
        plt.close()


if __name__ == '__main__':
    bigan = BIGAN()
    bigan.train(epochs=30, batch_size=64, sample_interval=200)


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 512)               51712     
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dense_6 (Dense)              (None, 512)               262656    
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 512)               2048      
_________________________________________________________________
dense_7 (Dense)              (None, 784)               402192    
__________

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 step:1 [D loss: 0.959536, acc: 29.69%] [G loss: 4.626052]
epoch:0 step:2 [D loss: 0.595189, acc: 62.50%] [G loss: 5.081646]
epoch:0 step:3 [D loss: 0.227627, acc: 93.75%] [G loss: 5.421597]
epoch:0 step:4 [D loss: 0.162079, acc: 98.44%] [G loss: 6.341406]
epoch:0 step:5 [D loss: 0.088884, acc: 99.22%] [G loss: 7.642174]
epoch:0 step:6 [D loss: 0.060217, acc: 100.00%] [G loss: 9.106062]
epoch:0 step:7 [D loss: 0.045664, acc: 99.22%] [G loss: 9.720157]
epoch:0 step:8 [D loss: 0.031582, acc: 100.00%] [G loss: 10.113420]
epoch:0 step:9 [D loss: 0.028420, acc: 100.00%] [G loss: 10.601333]
epoch:0 step:10 [D loss: 0.017493, acc: 100.00%] [G loss: 11.563252]
epoch:0 step:11 [D loss: 0.016290, acc: 100.00%] [G loss: 11.591646]
epoch:0 step:12 [D loss: 0.014904, acc: 100.00%] [G loss: 11.569134]
epoch:0 step:13 [D loss: 0.012463, acc: 100.00%] [G loss: 11.921530]
epoch:0 step:14 [D loss: 0.012938, acc: 100.00%] [G loss: 12.236393]
epoch:0 step:15 [D loss: 0.013923, acc: 100.00%] [G loss

epoch:0 step:121 [D loss: 0.149015, acc: 93.75%] [G loss: 9.243011]
epoch:0 step:122 [D loss: 0.210772, acc: 92.19%] [G loss: 9.750341]
epoch:0 step:123 [D loss: 0.254992, acc: 89.06%] [G loss: 9.830744]
epoch:0 step:124 [D loss: 0.218075, acc: 88.28%] [G loss: 9.230008]
epoch:0 step:125 [D loss: 0.287851, acc: 84.38%] [G loss: 10.812195]
epoch:0 step:126 [D loss: 0.199843, acc: 89.06%] [G loss: 8.976267]
epoch:0 step:127 [D loss: 0.245305, acc: 89.84%] [G loss: 10.378321]
epoch:0 step:128 [D loss: 0.223550, acc: 90.62%] [G loss: 9.636259]
epoch:0 step:129 [D loss: 0.243253, acc: 89.84%] [G loss: 10.219021]
epoch:0 step:130 [D loss: 0.331619, acc: 85.16%] [G loss: 9.974600]
epoch:0 step:131 [D loss: 0.206074, acc: 87.50%] [G loss: 9.873271]
epoch:0 step:132 [D loss: 0.171571, acc: 92.97%] [G loss: 9.953037]
epoch:0 step:133 [D loss: 0.232923, acc: 84.38%] [G loss: 9.985651]
epoch:0 step:134 [D loss: 0.186726, acc: 92.19%] [G loss: 9.248980]
epoch:0 step:135 [D loss: 0.371629, acc: 83.5

...

epoch:29 step:27556 [D loss: 0.694960, acc: 58.59%] [G loss: 1.708143]
epoch:29 step:27557 [D loss: 0.671913, acc: 64.06%] [G loss: 1.933252]
epoch:29 step:27558 [D loss: 0.607793, acc: 64.06%] [G loss: 1.916514]
epoch:29 step:27559 [D loss: 0.657526, acc: 57.03%] [G loss: 1.689690]
epoch:29 step:27560 [D loss: 0.662808, acc: 60.94%] [G loss: 1.811866]
epoch:29 step:27561 [D loss: 0.653935, acc: 60.94%] [G loss: 1.886207]
epoch:29 step:27562 [D loss: 0.638624, acc: 61.72%] [G loss: 1.742035]
epoch:29 step:27563 [D loss: 0.638014, acc: 63.28%] [G loss: 1.940146]
epoch:29 step:27564 [D loss: 0.683142, acc: 54.69%] [G loss: 1.754909]
epoch:29 step:27565 [D loss: 0.617646, acc: 67.19%] [G loss: 1.758259]
epoch:29 step:27566 [D loss: 0.651936, acc: 61.72%] [G loss: 1.807390]
epoch:29 step:27567 [D loss: 0.652844, acc: 60.94%] [G loss: 1.866590]
epoch:29 step:27568 [D loss: 0.656422, acc: 62.50%] [G loss: 1.896402]
epoch:29 step:27569 [D loss: 0.715814, acc: 47.66%] [G loss: 1.713464]
epoch:

epoch:29 step:27676 [D loss: 0.619771, acc: 64.06%] [G loss: 1.722479]
epoch:29 step:27677 [D loss: 0.615494, acc: 71.09%] [G loss: 1.997448]
epoch:29 step:27678 [D loss: 0.677743, acc: 60.94%] [G loss: 1.777662]
epoch:29 step:27679 [D loss: 0.666651, acc: 58.59%] [G loss: 1.762503]
epoch:29 step:27680 [D loss: 0.662669, acc: 60.16%] [G loss: 1.885162]
epoch:29 step:27681 [D loss: 0.612258, acc: 65.62%] [G loss: 1.937954]
epoch:29 step:27682 [D loss: 0.660713, acc: 67.97%] [G loss: 1.826506]
epoch:29 step:27683 [D loss: 0.629857, acc: 61.72%] [G loss: 1.786390]
epoch:29 step:27684 [D loss: 0.691271, acc: 61.72%] [G loss: 1.792895]
epoch:29 step:27685 [D loss: 0.631878, acc: 61.72%] [G loss: 1.871023]
epoch:29 step:27686 [D loss: 0.634203, acc: 64.06%] [G loss: 1.721989]
epoch:29 step:27687 [D loss: 0.631518, acc: 63.28%] [G loss: 1.950737]
epoch:29 step:27688 [D loss: 0.631643, acc: 59.38%] [G loss: 1.817553]
epoch:29 step:27689 [D loss: 0.659848, acc: 61.72%] [G loss: 2.037462]
epoch:

epoch:29 step:27796 [D loss: 0.624827, acc: 66.41%] [G loss: 1.827255]
epoch:29 step:27797 [D loss: 0.588632, acc: 74.22%] [G loss: 1.968207]
epoch:29 step:27798 [D loss: 0.667510, acc: 55.47%] [G loss: 1.848832]
epoch:29 step:27799 [D loss: 0.664269, acc: 62.50%] [G loss: 1.933651]
epoch:29 step:27800 [D loss: 0.648175, acc: 60.94%] [G loss: 1.832149]
epoch:29 step:27801 [D loss: 0.701065, acc: 57.03%] [G loss: 1.785232]
epoch:29 step:27802 [D loss: 0.647296, acc: 59.38%] [G loss: 1.845552]
epoch:29 step:27803 [D loss: 0.670272, acc: 54.69%] [G loss: 1.821207]
epoch:29 step:27804 [D loss: 0.677173, acc: 57.03%] [G loss: 1.822681]
epoch:29 step:27805 [D loss: 0.658550, acc: 60.94%] [G loss: 1.947776]
epoch:29 step:27806 [D loss: 0.645080, acc: 67.97%] [G loss: 1.963593]
epoch:29 step:27807 [D loss: 0.616299, acc: 71.88%] [G loss: 1.970921]
epoch:29 step:27808 [D loss: 0.604912, acc: 67.97%] [G loss: 1.931005]
epoch:29 step:27809 [D loss: 0.659173, acc: 60.94%] [G loss: 1.863110]
epoch:

epoch:29 step:27916 [D loss: 0.727275, acc: 57.03%] [G loss: 1.823751]
epoch:29 step:27917 [D loss: 0.685455, acc: 63.28%] [G loss: 1.774953]
epoch:29 step:27918 [D loss: 0.670913, acc: 58.59%] [G loss: 1.848057]
epoch:29 step:27919 [D loss: 0.636473, acc: 67.97%] [G loss: 1.904102]
epoch:29 step:27920 [D loss: 0.640877, acc: 64.84%] [G loss: 1.806445]
epoch:29 step:27921 [D loss: 0.620824, acc: 60.16%] [G loss: 1.711304]
epoch:29 step:27922 [D loss: 0.637769, acc: 64.06%] [G loss: 1.958315]
epoch:29 step:27923 [D loss: 0.641250, acc: 64.84%] [G loss: 1.814584]
epoch:29 step:27924 [D loss: 0.658369, acc: 56.25%] [G loss: 1.824431]
epoch:29 step:27925 [D loss: 0.694767, acc: 55.47%] [G loss: 1.711496]
epoch:29 step:27926 [D loss: 0.695235, acc: 54.69%] [G loss: 1.668809]
epoch:29 step:27927 [D loss: 0.624093, acc: 64.06%] [G loss: 1.795110]
epoch:29 step:27928 [D loss: 0.621550, acc: 71.09%] [G loss: 1.870742]
epoch:29 step:27929 [D loss: 0.686892, acc: 53.12%] [G loss: 1.801232]
epoch:

epoch:29 step:28036 [D loss: 0.638601, acc: 64.06%] [G loss: 1.842610]
epoch:29 step:28037 [D loss: 0.633748, acc: 66.41%] [G loss: 1.825848]
epoch:29 step:28038 [D loss: 0.718367, acc: 52.34%] [G loss: 1.708552]
epoch:29 step:28039 [D loss: 0.632012, acc: 64.84%] [G loss: 1.789893]
epoch:29 step:28040 [D loss: 0.647827, acc: 64.84%] [G loss: 1.829101]
epoch:29 step:28041 [D loss: 0.693077, acc: 54.69%] [G loss: 1.795893]
epoch:29 step:28042 [D loss: 0.658061, acc: 60.94%] [G loss: 1.799752]
epoch:29 step:28043 [D loss: 0.662117, acc: 53.91%] [G loss: 1.891876]
epoch:29 step:28044 [D loss: 0.708735, acc: 47.66%] [G loss: 1.846421]
epoch:29 step:28045 [D loss: 0.687001, acc: 56.25%] [G loss: 1.745193]
epoch:29 step:28046 [D loss: 0.651286, acc: 62.50%] [G loss: 1.802354]
epoch:29 step:28047 [D loss: 0.667607, acc: 58.59%] [G loss: 1.789759]
epoch:29 step:28048 [D loss: 0.596862, acc: 67.97%] [G loss: 1.844720]
epoch:29 step:28049 [D loss: 0.643440, acc: 64.84%] [G loss: 1.847373]
epoch: